In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt

In [2]:
import tensorflow as tf

In [3]:
train_data=pd.read_csv(r"C:\Users\bk\Documents\Projects\Machine Learning\Kaggle\NLP Disaster Tweets\nlp-getting-started(1)\train.csv")
test_data=pd.read_csv(r"C:\Users\bk\Documents\Projects\Machine Learning\Kaggle\NLP Disaster Tweets\nlp-getting-started(1)\test.csv")

In [4]:
train_data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
test_data.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [6]:
len(train_data)

7613

In [7]:
len(train_data.loc[train_data.target==1])

3271

In [8]:
from nltk.corpus import stopwords

In [9]:
longest_idx=train_data.text.str.len().idxmax()

In [10]:
shortest_idx=train_data.text.str.len().idxmin()

In [11]:
print(train_data['text'][longest_idx])
print(train_data['target'][longest_idx])
print("shortest")
print(train_data['text'][shortest_idx])
print(train_data['target'][shortest_idx])

when you're taking a shower and someone flushes the toilet and you have .1 second to GTFO or you get burned??????????????????????????????????????????????????
0
shortest
Crushed
0


In [12]:
train_text=train_data.text.values
train_text

array(['Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
       'Forest fire near La Ronge Sask. Canada',
       "All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
       ...,
       'M1.94 [01:04 UTC]?5km S of Volcano Hawaii. http://t.co/zDtoyd8EbJ',
       'Police investigating after an e-bike collided with a car in Little Portugal. E-bike rider suffered serious non-life threatening injuries.',
       'The Latest: More Homes Razed by Northern California Wildfire - ABC News http://t.co/YmY4rSkQ3d'],
      dtype=object)

In [13]:
train_target=train_data.target.values

In [14]:
test_text=test_data.text.values

In [15]:
def remove_stopword(training_data):
    for k in range(len(training_data)):

        sent=training_data[k]
        sent=sent.lower()
        words=sent.split()
        wordsa=[w for w in words if w not in stopwords.words('english')]
        sent=" ".join(wordsa)
        training_data[k]=sent
    return training_data

In [16]:
train_text=remove_stopword(train_text)

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
train_texts, test_texts, train_label, test_label=train_test_split(train_text, train_target)

In [19]:
#hyperparameters

NUM_WORDS=1000
OOV_TOKEN="<OOV>"
MAXLEN=120
PADDING='post'
EMBEDDING_DIM=128
EPOCHS=100
BATCH_SIZE=32

def fit_tokenizer(train_textss, num_words, oov_token):
    tokenizer=Tokenizer(num_words=num_words,oov_token=oov_token)
    tokenizer.fit_on_texts(train_textss)
    return tokenizer

tokenizer=fit_tokenizer(train_texts, NUM_WORDS, OOV_TOKEN)
word_index=tokenizer.word_index

In [20]:
word_index

{'<OOV>': 1,
 't': 2,
 'co': 3,
 'http': 4,
 'https': 5,
 'amp': 6,
 'like': 7,
 "i'm": 8,
 'fire': 9,
 'via': 10,
 'new': 11,
 '2': 12,
 'get': 13,
 'news': 14,
 'one': 15,
 'people': 16,
 "'": 17,
 'video': 18,
 'emergency': 19,
 'police': 20,
 'disaster': 21,
 'would': 22,
 'u': 23,
 'still': 24,
 '3': 25,
 'back': 26,
 'burning': 27,
 'storm': 28,
 'got': 29,
 'time': 30,
 'body': 31,
 'day': 32,
 'california': 33,
 'man': 34,
 '1': 35,
 'first': 36,
 'dead': 37,
 'us': 38,
 'rt': 39,
 'know': 40,
 'see': 41,
 'suicide': 42,
 'going': 43,
 'crash': 44,
 'year': 45,
 'bomb': 46,
 'youtube': 47,
 "can't": 48,
 'buildings': 49,
 '4': 50,
 'world': 51,
 'killed': 52,
 'nuclear': 53,
 'fires': 54,
 'families': 55,
 'today': 56,
 'two': 57,
 'war': 58,
 'full': 59,
 'life': 60,
 'car': 61,
 'love': 62,
 'attack': 63,
 'train': 64,
 '2015': 65,
 'go': 66,
 'many': 67,
 'say': 68,
 'gt': 69,
 '5': 70,
 's': 71,
 'old': 72,
 'may': 73,
 'way': 74,
 'think': 75,
 'could': 76,
 'good': 77,
 '

In [21]:
def seq_and_pad(texts, tokenizer, padding, maxlen):
    sequences=tokenizer.texts_to_sequences(texts)
    padded_sequences=pad_sequences(sequences, maxlen=maxlen, padding=padding)
    return padded_sequences


In [22]:
train_padded_seq=seq_and_pad(train_texts, tokenizer, PADDING, MAXLEN)
test_padded_seq=seq_and_pad(test_texts, tokenizer, PADDING, MAXLEN)

In [23]:
early_stopping=tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

In [24]:
def create_model(num_words, embedding_dim, maxlen):
    tf.random.set_seed(123)
    model=tf.keras.Sequential([tf.keras.layers.Embedding(num_words, embedding_dim, input_length=maxlen),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='RMSprop', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [25]:
model=create_model(NUM_WORDS, EMBEDDING_DIM, MAXLEN)

In [26]:
history=model.fit(train_padded_seq, train_label, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(test_padded_seq, test_label), callbacks=[early_stopping])

Epoch 1/100
179/179 [==============================] - 12s 11ms/step - loss: 0.6837 - accuracy: 0.5705 - val_loss: 0.6842 - val_accuracy: 0.5646
Epoch 2/100
179/179 [==============================] - 1s 8ms/step - loss: 0.6830 - accuracy: 0.5719 - val_loss: 0.6832 - val_accuracy: 0.5646
Epoch 3/100
179/179 [==============================] - 1s 8ms/step - loss: 0.6791 - accuracy: 0.5723 - val_loss: 0.6793 - val_accuracy: 0.5646
Epoch 4/100
179/179 [==============================] - 1s 8ms/step - loss: 0.6727 - accuracy: 0.5681 - val_loss: 0.6661 - val_accuracy: 0.5562
Epoch 5/100
179/179 [==============================] - 1s 8ms/step - loss: 0.6501 - accuracy: 0.6073 - val_loss: 0.6517 - val_accuracy: 0.6266
Epoch 6/100
179/179 [==============================] - 1s 8ms/step - loss: 0.6374 - accuracy: 0.6390 - val_loss: 0.6361 - val_accuracy: 0.6276
Epoch 7/100
179/179 [==============================] - 1s 8ms/step - loss: 0.6300 - accuracy: 0.6534 - val_loss: 0.6312 - val_accuracy: 0.64

In [27]:
test_pred=remove_stopword(test_text)

In [28]:
test_padded=seq_and_pad(test_pred, tokenizer, PADDING, MAXLEN)

In [29]:
test_padded

array([[935,   1,  61, ...,   0,   0,   0],
       [408, 177, 934, ...,   0,   0,   0],
       [110,   9, 651, ...,   0,   0,   0],
       ...,
       [970, 635, 296, ...,   0,   0,   0],
       [  1, 387, 288, ...,   0,   0,   0],
       [  1,   1,   1, ...,   0,   0,   0]])

In [30]:
testpred=model.predict(test_padded)

102/102 [==============================] - 1s 3ms/step


In [31]:
testpred

array([[0.30246305],
       [0.57049537],
       [0.71279794],
       ...,
       [0.5640656 ],
       [0.4264142 ],
       [0.05811455]], dtype=float32)

In [32]:


testpred=testpred.round().astype(int)
testpred=testpred.tolist()

In [33]:
submission=pd.DataFrame({"id":test_data.id, "target":testpred})

In [34]:
submission['target']=testpred

In [35]:
submission

,id,target
0,0,[0]
1,2,[1]
2,3,[1]
3,9,[0]
4,11,[1]
...,...,...
3258,10861,[0]
3259,10865,[1]
3260,10868,[1]
3261,10874,[0]


In [42]:
submission['target']=submission['target'].astype(str).str.strip('[|]')

In [43]:
submission

,id,target
0,0,0
1,2,1
2,3,1
3,9,0
4,11,1
...,...,...
3258,10861,0
3259,10865,1
3260,10868,1
3261,10874,0


In [45]:
submission.to_csv(r"C:\Users\bk\Documents\Projects\Machine Learning\Kaggle\NLP Disaster Tweets\submission.csv", index=False)